In [132]:
# Imports
import pandas as pd
import altair as alt

# Loading the data
url = 'https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB'
sessions = pd.read_csv(url)
sessions = sessions.dropna()

In [133]:
# Dropping unnecessary columns
sessions_drop = sessions.drop(columns=['hashedEmail','original_start_time','original_end_time'])

In [139]:
# Creating a new column for day of the week
sessions_drop['start_time'] = pd.to_datetime(sessions['start_time'], format='%d/%m/%Y %H:%M')
sessions_drop['end_time'] = pd.to_datetime(sessions['end_time'], format='%d/%m/%Y %H:%M')
sessions_days = sessions_drop.assign(day_of_week = sessions_drop['start_time'].dt.dayofweek)
# Renaming the values in the day_of_week column
sessions_days['day_of_week'] = sessions_days['day_of_week'].replace({
    0:'Monday',
    1:'Tuesday',
    2:'Wednesday',
    3:'Thursday',
    4:'Friday',
    5:'Saturday',
    6:'Sunday'
})

In [140]:
# Calculate the duration of each access in minutes
sessions_days['start_time'] = pd.to_datetime(sessions_days['start_time'], format='%d/%m/%Y %H:%M')
sessions_days['end_time'] = pd.to_datetime(sessions_days['end_time'], format='%d/%m/%Y %H:%M')
sessions_days['duration_minutes'] = (sessions_days['end_time'] - sessions_days['start_time']) / pd.Timedelta(minutes=1)
sessions_days

,start_time,end_time,day_of_week,duration_minutes
0,2024-06-30 18:12:00,2024-06-30 18:24:00,Sunday,12.0
1,2024-06-17 23:33:00,2024-06-17 23:46:00,Monday,13.0
2,2024-07-25 17:34:00,2024-07-25 17:57:00,Thursday,23.0
3,2024-07-25 03:22:00,2024-07-25 03:58:00,Thursday,36.0
4,2024-05-25 16:01:00,2024-05-25 16:12:00,Saturday,11.0
...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,Friday,6.0
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,Monday,11.0
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,Sunday,21.0
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,Thursday,7.0


In [141]:
# Calculate the average duration
avg_duration = sessions_days.groupby('day_of_week')['duration_minutes'].mean().reset_index()
# Make a plot 
week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
duration_plot = alt.Chart(avg_duration).mark_bar().encode(
    x=alt.X('duration_minutes:Q').title('Average duration of each session(minutes)'),
    y=alt.Y('day_of_week:N').title('Day of week').sort(week_order),
    color=alt.Color('day_of_week:N')
)
duration_plot                         

alt.Chart(...)

In [200]:
# Converting the data into means of hours
sessions_days['start_time'] = pd.to_datetime(sessions['start_time'], dayfirst=True).dt.hour.astype(int)
sessions_days['end_time'] = pd.to_datetime(sessions['end_time'], dayfirst=True).dt.hour.astype(int)
# Extract the sessions where the start and end have different hours
different_sessions = sessions_days[sessions_days['start_time'] != sessions_days['end_time']]
# Rename the end_time column to start_time to vertically stack the data
different_sessions = different_sessions[['end_time', 'day_of_week']].rename(columns={'end_time':'start_time'})
# Combine the data to ensure the data contains more data and is more accurate in terms of the number of users accessing
sessions_new = sessions_days[['start_time', 'day_of_week']]
sessions_combined = pd.concat([sessions_new, different_sessions], ignore_index=True)
sessions_combined = sessions_combined.rename(columns={'start_time':'session_time'})
# Groupby and count
sessions_count = sessions_combined.groupby('day_of_week')['session_time'].value_counts().reset_index()
sessions_chart = alt.Chart(sessions_count).mark_line().encode(
    x=alt.X('session_time:Q').title('Hour'),
    y=alt.Y('count:Q').title('Number of users accessing')
).facet(column=alt.Column('day_of_week:N', sort=week_order))
sessions_chart

alt.FacetChart(...)

### 1. Data description
#### Sessions dataset
- Number of observations:  1535
- Number of variables/columns:  5
- Variables:

| Variable Name         | Type        | Description                                                                 |
|-----------------------|-------------|------------------------------------------------------------------------------|
| hashedEmail           | Categorical | Unique player identification                                                |
| start_time            | Temporal    | Timestamp of when player logs in (day/month/year/hour)                      |
| end_time              | Temporal    | Timestamp of when player logs off (day/month/year/hour)                     |
| original_start_time   | Temporal    | Exact second player logged on to the server since the server was set up                     |
| original_end_time     | Temporal    | Exact second player logged off the server since the server was set up               |
